# 이미지 분류(Image Classification) 작업 파이프라인
- 학습 data '38-cloud' 데이터셋 확보
- 데이터 확인 및 전처리
- classification model "VG
- 모델 파라미터 정의
- 모델 학습

- 런타임: Colab 또는 Jupyter(+GPU 권장)
- 필요한 패키지 설치: torch/torchvision, opencv, sklearn, pandas 등


In [1]:
!nvidia-smi -L || true

# PyTorch (Colab CUDA 11.8 예시)
!pip -q install torch torchvision --index-url https://download.pytorch.org/whl/cu118

# 일반 유틸
!pip -q install opencv-python matplotlib scikit-learn pandas tqdm


/bin/bash: line 1: nvidia-smi: command not found


# 데이터셋 다운로드
데이터셋 정보: NUAA-CR4L8/9 (Zenodo)

설명: Landsat 8/9 영상에서 동일 지역을 8일 간격으로 촬영한 쌍—하나는 구름이 있는 영상, 다른 하나는 구름이 없는 영상으로 구성. 전체 20쌍(16쌍은 훈련, 4쌍은 테스트)

In [2]:
# 디렉토리 먼저 생성
!mkdir -p data

# 실제 파일 다운로드 (예시 URL)
!wget -O NUAA_CR4L8_9.zip "https://zenodo.org/records/15892748/files/NUAA-CR4L89.zip"

# 압축 해제
!unzip NUAA_CR4L8_9.zip -d data/NUAA_CR4L8_9


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
file #742:  bad zipfile offset (local header sig):  1424086207
file #743:  bad zipfile offset (local header sig):  1426113912
file #744:  bad zipfile offset (local header sig):  1428029333
file #745:  bad zipfile offset (local header sig):  1430021045
file #746:  bad zipfile offset (local header sig):  1432049741
file #747:  bad zipfile offset (local header sig):  1434016308
file #748:  bad zipfile offset (local header sig):  1435986717
file #749:  bad zipfile offset (local header sig):  1437997201
file #750:  bad zipfile offset (local header sig):  1440079415
file #751:  bad zipfile offset (local header sig):  1442081139
file #752:  bad zipfile offset (local header sig):  1444032702
file #753:  bad zipfile offset (local header sig):  1446018239
file #754:  bad zipfile offset (local header sig):  1447974285
file #755:  bad zipfile offset (local header sig):  1450013277
file #756:  bad zipfile offset (local header sig):  1452040001
file #757:  bad zip

In [9]:
import os, re, shutil, pathlib
import glob

src_base = "data/NUAA_CR4L8_9"
dst_base = "data/classify"

# Remove existing destination directory to start fresh
shutil.rmtree(dst_base, ignore_errors=True)

# Define keyword patterns for clear and cloudy images, being more flexible
# Look for keywords in the parent directory names as well
cloudy_kw = re.compile(r"(cloudy|cloud|CLD)(?!.*(?:free|less))", re.IGNORECASE)
clear_kw  = re.compile(r"(clear|cloud[-_ ]?free|cloud[-_ ]?less|no[-_ ]?cloud|GT)", re.IGNORECASE)
valid_ext = (".tif",".tiff",".png",".jpg",".jpeg",".bmp",".webp")

# Create destination directories
for split in ["train","test"]:
    os.makedirs(os.path.join(dst_base, split, "cloudy"), exist_ok=True)
    os.makedirs(os.path.join(dst_base, split, "clear"),  exist_ok=True)

# Function to determine label from file path
def get_label(filepath):
    # Check parent directory names for clues
    path_parts = pathlib.Path(filepath).parts
    for part in reversed(path_parts):
        if clear_kw.search(part):
            return "clear"
        elif cloudy_kw.search(part):
            return "cloudy"

    # Fallback to filename if parent directories don't provide a clear label
    filename = os.path.basename(filepath).lower()
    if clear_kw.search(filename):
        return "clear"
    elif cloudy_kw.search(filename):
        return "cloudy"

    return None


# Iterate through the downloaded files and create symbolic links
linked_count = {"clear": 0, "cloudy": 0}
error_count = 0

for split in ["train", "test"]:
    split_dir = os.path.join(src_base, split)
    if split == "train":
        # Special handling for the training set based on observed structure
        train_gt_dir = os.path.join(split_dir, "GT")
        if os.path.isdir(train_gt_dir):
            for filepath in glob.glob(os.path.join(train_gt_dir, "**", "*"), recursive=True):
                if not os.path.isfile(filepath):
                    continue
                ext = os.path.splitext(filepath)[1].lower()
                if ext not in valid_ext:
                    continue
                label = "clear" # Assume all images in GT are clear
                try:
                    relative_path = os.path.relpath(filepath, src_base)
                    dst_name = relative_path.replace(os.sep, "_").lower()
                    dst_path = os.path.join(dst_base, split, label, dst_name)

                    if not os.path.exists(dst_path):
                        try:
                            pathlib.Path(dst_path).symlink_to(os.path.abspath(filepath))
                            linked_count[label] += 1
                        except Exception as e:
                            print(f"Symlink failed for {filepath}, copying instead. Error: {e}")
                            shutil.copy2(filepath, dst_path)
                            linked_count[label] += 1
                except Exception as e:
                    print(f"Error processing file {filepath}: {e}")
                    error_count += 1
    else: # Handle test set with the original logic
        for filepath in glob.glob(os.path.join(split_dir, "**", "*"), recursive=True):
            if not os.path.isfile(filepath):
                continue
            ext = os.path.splitext(filepath)[1].lower()
            if ext not in valid_ext:
                continue

            label = get_label(filepath)
            if label:
                try:
                    relative_path = os.path.relpath(filepath, src_base)
                    dst_name = relative_path.replace(os.sep, "_").lower()
                    dst_path = os.path.join(dst_base, split, label, dst_name)

                    if not os.path.exists(dst_path):
                        try:
                            pathlib.Path(dst_path).symlink_to(os.path.abspath(filepath))
                            linked_count[label] += 1
                        except Exception as e:
                            print(f"Symlink failed for {filepath}, copying instead. Error: {e}")
                            shutil.copy2(filepath, dst_path)
                            linked_count[label] += 1
                except Exception as e:
                    print(f"Error processing file {filepath}: {e}")
                    error_count += 1


# Report results
print("\n--- Processing Complete ---")
print(f"Successfully processed files: clear={linked_count['clear']}, cloudy={linked_count['cloudy']}")
print(f"Files with processing errors: {error_count}")

# Final count in destination directories
print("\n--- Final Counts in Destination ---")
for split in ["train","test"]:
    clear_count = len(os.listdir(os.path.join(dst_base, split, "clear")))
    cloudy_count = len(os.listdir(os.path.join(dst_base, split, "cloudy")))
    print(f"[{split}] clear={clear_count}  cloudy={cloudy_count}")


--- Processing Complete ---
Successfully processed files: clear=663, cloudy=549
Files with processing errors: 0

--- Final Counts in Destination ---
[train] clear=114  cloudy=0
[test] clear=549  cloudy=549


## 데이터 구조 확인
압축 해제 후 데이터 구조는 예상과 다를 수 있습니다. 현재 확인된 구조는 `train/` 안에 `GT` 폴더만 있고, 그 안에 숫자로 된 서브폴더들이 있으며, 실제 이미지 파일(`.tif`, `.png` 등)은 그 서브폴더 안에 있습니다. `test/` 안에는 `GT`, `Cloud`, `CLD` 등의 폴더가 있고, 그 안에 숫자로 된 서브폴더들이 있으며, 실제 이미지 파일(`.tif`, `.png` 등)은 그 서브폴더 안에 있습니다. 'clear' 이미지는 주로 'GT' 폴더 안에, 'cloudy' 이미지는 'Cloud' 또는 'CLD' 폴더 안에 있는 것으로 파악됩니다.

In [8]:
import os

# List contents of the train directory
train_dir_contents = os.listdir("data/NUAA_CR4L8_9/train")
print(f"Contents of data/NUAA_CR4L8_9/train: {train_dir_contents}")

Contents of data/NUAA_CR4L8_9/train: ['GT']


## 데이터 구조 확인
압축 해제 후 데이터 구조는 예상과 다를 수 있습니다. 현재 확인된 구조는 `train/` 또는 `test/` 안에 `GT`, `Cloud`, `CLD` 등의 폴더가 있고, 그 안에 숫자로 된 서브폴더들이 있으며, 실제 이미지 파일(`.tif`, `.png` 등)은 그 서브폴더 안에 있습니다. 'clear' 이미지는 주로 'GT' 폴더 안에, 'cloudy' 이미지는 'Cloud' 또는 'CLD' 폴더 안에 있는 것으로 파악됩니다.

In [4]:
import os

# List contents of a sample pair directory in the train set
sample_pair_dir = os.path.join("data/NUAA_CR4L8_9", "train", os.listdir(os.path.join("data/NUAA_CR4L8_9", "train"))[0])
print(f"Contents of {sample_pair_dir}:")
print(os.listdir(sample_pair_dir))

Contents of data/NUAA_CR4L8_9/train/GT:
['028033', '028032']


## 데이터 구조 확인
압축을 풀면 train/, test/ 안에 pair_XXXX 폴더들이 있고, 각각 cloudy.tif, clear.tif가 있을 거예요.

In [ ]:
import os

base = "data/NUAA_CR4L8_9"
for split in ["train","test"]:
    path = os.path.join(base, split)
    print(f"[{split}] {len(os.listdir(path))} pairs")
    sample_pair = os.listdir(path)[0]
    print(" sample files:", os.listdir(os.path.join(path, sample_pair)))


[train] 1 pairs
 sample files: ['028033', '028032']
[test] 2 pairs
 sample files: ['028031', '023037', '017035', '028035']


# 과제
"/content/data/NUAA_CR4L8_9"의 데이터 구조를 분석하고, 선택한 셀의 코드를 재정의하여 데이터를 올바르게 로드하고, 이미지 분류 모델을 학습하여 선명한 이미지와 흐린 이미지를 구분합니다.

## 데이터 분포 평가

### 하위 작업:
테스트 세트에서 사용 가능한 선명한 이미지와 흐린 이미지의 정확한 수를 확인하여 데이터 균형을 파악합니다.


**추론**:
테스트 세트 내의 투명 디렉토리와 흐린 디렉토리에 있는 파일 수를 세고 결과를 인쇄합니다.



In [11]:
import os

clear_test_count = len(os.listdir("data/classify/test/clear"))
cloudy_test_count = len(os.listdir("data/classify/test/cloudy"))

print(f"Number of clear images in test set: {clear_test_count}")
print(f"Number of cloudy images in test set: {cloudy_test_count}")

Number of clear images in test set: 549
Number of cloudy images in test set: 549


## 테스트 데이터 분할

### 하위 작업:
테스트 데이터 세트를 새 학습 세트(선명한 이미지와 흐린 이미지 모두 포함)와 유효성 검사 세트로 분할합니다.


**추론**:
필요한 라이브러리를 가져오고 디렉토리 경로를 정의합니다. 각각의 클래스 하위 디렉터리가 있는 새 교육 및 유효성 검사 디렉터리를 만듭니다.



In [12]:
import os
import shutil
import random

src_test_dir = "data/classify/test"
dst_train_dir = "data/classify/new_train"
dst_val_dir = "data/classify/val"

# Remove existing destination directories to start fresh
shutil.rmtree(dst_train_dir, ignore_errors=True)
shutil.rmtree(dst_val_dir, ignore_errors=True)

# Create new training and validation directories
for split_dir in [dst_train_dir, dst_val_dir]:
    os.makedirs(os.path.join(split_dir, "clear"), exist_ok=True)
    os.makedirs(os.path.join(split_dir, "cloudy"), exist_ok=True)

**추론**:
이미지 파일을 나열하고, 섞어서 새로운 학습 및 검증 세트로 분할한 다음 파일을 복사/이동합니다. 마지막으로 새 디렉토리에 개수를 인쇄합니다.



In [13]:
# List image files
clear_files = [os.path.join(src_test_dir, "clear", f) for f in os.listdir(os.path.join(src_test_dir, "clear"))]
cloudy_files = [os.path.join(src_test_dir, "cloudy", f) for f in os.listdir(os.path.join(src_test_dir, "cloudy"))]

# Shuffle files
random.shuffle(clear_files)
random.shuffle(cloudy_files)

# Determine split ratio (e.g., 80% train, 20% validation)
split_ratio = 0.8
clear_train_split = int(len(clear_files) * split_ratio)
cloudy_train_split = int(len(cloudy_files) * split_ratio)

# Split and copy/move files
for i, file_path in enumerate(clear_files):
    if i < clear_train_split:
        dest_dir = os.path.join(dst_train_dir, "clear")
    else:
        dest_dir = os.path.join(dst_val_dir, "clear")
    shutil.copy(file_path, dest_dir)

for i, file_path in enumerate(cloudy_files):
    if i < cloudy_train_split:
        dest_dir = os.path.join(dst_train_dir, "cloudy")
    else:
        dest_dir = os.path.join(dst_val_dir, "cloudy")
    shutil.copy(file_path, dest_dir)

# Verify the split
print("\n--- New Dataset Split Counts ---")
for split_dir in [dst_train_dir, dst_val_dir]:
    split_name = os.path.basename(split_dir)
    clear_count = len(os.listdir(os.path.join(split_dir, "clear")))
    cloudy_count = len(os.listdir(os.path.join(split_dir, "cloudy")))
    print(f"[{split_name}] clear={clear_count}  cloudy={cloudy_count}")


--- New Dataset Split Counts ---
[new_train] clear=439  cloudy=439
[val] clear=110  cloudy=110


## 데이터로더 업데이트

### 하위 작업:
새로 생성된 학습 및 유효성 검사 집합을 사용하도록 데이터로더 구성을 수정합니다.


**Reasoning**:
Modify the DataLoader configuration to use the newly created training and validation sets and verify the data loading.



In [14]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

IMG_SIZE = 224
BATCH = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# Update the directories to point to the new split
train_dir = "data/classify/new_train"
val_dir = "data/classify/val" # Use val_dir for the validation set

# Instantiate ImageFolder datasets for the new split
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
val_ds  = datasets.ImageFolder(val_dir,  transform=eval_tfms) # Use val_ds for the validation set

# Print information to verify
print("Classes:", train_ds.classes)
print("New Train:", len(train_ds), " Validation:", len(val_ds))

Classes: ['clear', 'cloudy']
New Train: 878  Validation: 220


## 분류 모델 정의

### 하위 작업:
적합한 이미지 분류 모델을 선택하고 정의합니다(예: 토치비전의 사전 학습된 모델).


**추론**:
사전 학습된 분류 모델을 가져와 이진 분류를 위해 최종 레이어를 수정합니다.



In [15]:
import torch
import torch.nn as nn
import torchvision.models as models

# Check for GPU availability and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained ResNet18 model
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer to have 2 output features (for clear and cloudy)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the selected device
model = model.to(device)

print("Model loaded and modified successfully.")

Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


100%|██████████| 44.7M/44.7M [00:01<00:00, 37.1MB/s]


Model loaded and modified successfully.


## 손실 함수 및 옵티마이저 정의

### 하위 작업:
모델 학습을 위한 손실 함수와 최적화기를 설정합니다.


**추론**:
지침에 따라 모델 학습을 위한 손실 함수 및 최적화 프로그램을 설정합니다.



In [16]:
import torch.optim
import torch.nn as nn

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

print("Loss function and optimizer set up successfully.")

Loss function and optimizer set up successfully.


## 훈련 모델

### 하위 작업:
학습 루프를 구현하여 준비된 데이터에 대한 분류 모델을 학습합니다.




**추론**:
정의된 모델, 기준 및 최적화 도구를 사용하여 학습 및 검증 루프를 구현합니다.



In [17]:
import time

# Create DataLoaders
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

# Define the number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_ds)

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    val_loss = val_running_loss / len(val_ds)
    val_accuracy = correct_predictions / total_predictions

    end_time = time.time()
    epoch_duration = end_time - start_time

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

print("Training complete.")

ERROR:PIL.TiffImagePlugin:More samples per pixel than can be decoded: 10
ERROR:PIL.TiffImagePlugin:More samples per pixel than can be decoded: 10
ERROR:PIL.TiffImagePlugin:More samples per pixel than can be decoded: 10
ERROR:PIL.TiffImagePlugin:More samples per pixel than can be decoded: 10


UnidentifiedImageError: Caught UnidentifiedImageError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/worker.py", line 349, in _worker_loop
    data = fetcher.fetch(index)  # type: ignore[possibly-undefined]
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/_utils/fetch.py", line 52, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchvision/datasets/folder.py", line 245, in __getitem__
    sample = self.loader(path)
             ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchvision/datasets/folder.py", line 284, in default_loader
    return pil_loader(path)
           ^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/torchvision/datasets/folder.py", line 263, in pil_loader
    img = Image.open(f)
  

ERROR:PIL.TiffImagePlugin:More samples per pixel than can be decoded: 10


**추론**:
이전 시도는 `.tif` 파일을 로드할 때 `UnidentifiedImageError`로 인해 실패했는데, 이는 기본 PIL 로더에서 이미지 형식이나 속성을 완전히 지원하지 않아서 발생한 것 같습니다. 이 문제를 해결하기 위해 데이터 로더를 수정하여 OpenCV의 `imread`를 이미지 로더로 사용하도록 할 것입니다. OpenCV는 채널이 3개 이상인 잠재적으로 문제가 될 수 있는 TIFF 파일을 포함하여 더 광범위한 이미지 형식과 속성을 처리하는 경우가 많기 때문입니다. 또한 로딩 문제를 격리하기 위해 처음에는 `num_workers`를 0으로 유지한 후 다시 늘릴 수 있습니다.



In [18]:
import cv2
from PIL import Image
import numpy as np

# Define a custom loader function using OpenCV
def opencv_loader(path):
    # Read image with OpenCV
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

    # Check if image reading was successful
    if img is None:
        raise IOError(f"Cannot load image file: {path}")

    # OpenCV reads images as BGR, convert to RGB
    if len(img.shape) == 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    elif len(img.shape) == 2: # Grayscale image
         img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB by duplicating channels

    # Handle alpha channel if present (more than 3 channels)
    if img.shape[-1] > 3:
        img = img[:, :, :3] # Keep only the first 3 channels (RGB)

    # Convert numpy array to PIL Image
    img = Image.fromarray(img)

    return img


# Create DataLoaders using the custom loader and num_workers=0 initially
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=0, collate_fn=None)
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=0, collate_fn=None)

# Update the datasets to use the custom loader
train_ds.loader = opencv_loader
val_ds.loader = opencv_loader


# Define the number of epochs
num_epochs = 10

# Training loop
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_ds)

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    val_loss = val_running_loss / len(val_ds)
    val_accuracy = correct_predictions / total_predictions

    end_time = time.time()
    epoch_duration = end_time - start_time

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

print("Training complete.")

OSError: Cannot load image file: data/classify/new_train/cloudy/test_cloud_028035_496.tif

**추론**:
사용자 지정 OpenCV 로더가 여전히 특정 TIFF 파일을 로드하지 못하고 있으며, 이는 특정 이미지 형식이나 콘텐츠에 지속적인 문제가 있어 PIL이나 기본 OpenCV 읽기로는 처리할 수 없음을 나타냅니다. 시간 제약이 있고 훈련 루프 하위 작업을 완료하는 데 집중해야 한다는 점을 고려할 때 가장 실용적인 접근 방식은 데이터 세트에서 문제가 있는 파일을 식별하고 제외하는 것입니다. 사용자 정의 로더로 로딩 오류를 일으키는 파일을 필터링하도록 데이터 세트 생성을 수정하겠습니다.



In [19]:
import os
import shutil
import random
import cv2
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a custom loader function using OpenCV with error handling
def robust_opencv_loader(path):
    try:
        # Read image with OpenCV
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

        # Check if image reading was successful
        if img is None:
            print(f"Warning: Could not load image with OpenCV: {path}")
            return None

        # OpenCV reads images as BGR, convert to RGB
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif len(img.shape) == 2: # Grayscale image
             img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB by duplicating channels

        # Handle alpha channel if present (more than 3 channels)
        if img.shape[-1] > 3:
            img = img[:, :, :3] # Keep only the first 3 channels (RGB)

        # Convert numpy array to PIL Image
        img = Image.fromarray(img)

        return img
    except Exception as e:
        print(f"Warning: Error loading image {path} with OpenCV: {e}")
        return None


# --- Dataset Creation with Filtering ---
IMG_SIZE = 224
BATCH = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_dir = "data/classify/new_train"
val_dir = "data/classify/val"

# Load datasets and filter out unreadable images
train_ds_full = datasets.ImageFolder(train_dir)
val_ds_full = datasets.ImageFolder(val_dir)

# Filter function to check if an image can be loaded
def is_valid_image(filepath, loader):
    img = loader(filepath)
    return img is not None

# Get valid samples
train_samples = [(path, label) for path, label in train_ds_full.samples if is_valid_image(path, robust_opencv_loader)]
val_samples = [(path, label) for path, label in val_ds_full.samples if is_valid_image(path, robust_opencv_loader)]

# Create new datasets with only valid samples and apply transforms
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
train_ds.samples = train_samples
train_ds.imgs = train_samples # Update .imgs as well for compatibility

val_ds = datasets.ImageFolder(val_dir, transform=eval_tfms)
val_ds.samples = val_samples
val_ds.imgs = val_samples # Update .imgs as well for compatibility

# Set the custom loader for the filtered datasets
train_ds.loader = robust_opencv_loader
val_ds.loader = robust_opencv_loader


print("Classes:", train_ds.classes)
print("New Train (valid):", len(train_ds), " Validation (valid):", len(val_ds))

# --- Model, Criterion, Optimizer (re-define just in case) ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# --- DataLoaders ---
train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2) # Increased num_workers back
val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)   # Increased num_workers back

# Define the number of epochs
num_epochs = 10

# --- Training loop ---
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()  # Set model to training mode
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

    epoch_loss = running_loss / len(train_ds)

    # Validation phase
    model.eval()  # Set model to evaluation mode
    val_running_loss = 0.0
    correct_predictions = 0
    total_predictions = 0

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * inputs.size(0)

            _, predicted = torch.max(outputs.data, 1)
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

    val_loss = val_running_loss / len(val_ds)
    val_accuracy = correct_predictions / total_predictions

    end_time = time.time()
    epoch_duration = end_time - start_time

    print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

print("Training complete.")

Classes: ['clear', 'cloudy']
New Train (valid): 0  Validation (valid): 0
Using device: cpu


ValueError: num_samples should be a positive integer value, but got num_samples=0

**추론**:
"num_samples는 양의 정수 값이어야 하지만 num_samples=0을 얻었습니다."라는 오류는 필터링 후 데이터 세트 중 하나 또는 둘(train_ds 또는 val_ds)의 샘플이 0으로 끝났음을 나타냅니다. 이는 학습 또는 검증 분할의 모든 이미지를 강력한 OpenCV 로더가 읽을 수 없음을 의미합니다. 필터링 후 유효한 샘플 수를 확인하고 유효한 이미지가 남아 있지 않은 경우 접근 방식을 조정해야 합니다. 데이터로더를 생성하기 전에 `train_samples`와 `val_samples`의 길이를 인쇄하여 문제를 파악하겠습니다.



In [20]:
import os
import shutil
import random
import cv2
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Define a custom loader function using OpenCV with error handling
def robust_opencv_loader(path):
    try:
        # Read image with OpenCV
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

        # Check if image reading was successful
        if img is None:
            # print(f"Warning: Could not load image with OpenCV: {path}") # Suppress frequent warnings during filtering
            return None

        # OpenCV reads images as BGR, convert to RGB
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif len(img.shape) == 2: # Grayscale image
             img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB by duplicating channels

        # Handle alpha channel if present (more than 3 channels)
        if img.shape[-1] > 3:
            img = img[:, :, :3] # Keep only the first 3 channels (RGB)

        # Convert numpy array to PIL Image
        img = Image.fromarray(img)

        return img
    except Exception as e:
        # print(f"Warning: Error loading image {path} with OpenCV: {e}") # Suppress frequent warnings during filtering
        return None


# --- Dataset Creation with Filtering ---
IMG_SIZE = 224
BATCH = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_dir = "data/classify/new_train"
val_dir = "data/classify/val"

# Load datasets and filter out unreadable images
train_ds_full = datasets.ImageFolder(train_dir)
val_ds_full = datasets.ImageFolder(val_dir)

# Filter function to check if an image can be loaded
def is_valid_image(filepath, loader):
    img = loader(filepath)
    return img is not None

# Get valid samples
print("Filtering training images...")
train_samples = [(path, label) for path, label in train_ds_full.samples if is_valid_image(path, robust_opencv_loader)]
print("Filtering validation images...")
val_samples = [(path, label) for path, label in val_ds_full.samples if is_valid_image(path, robust_opencv_loader)]

# Print the number of valid samples
print(f"Number of valid training samples: {len(train_samples)}")
print(f"Number of valid validation samples: {len(val_samples)}")


# Create new datasets with only valid samples and apply transforms
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
train_ds.samples = train_samples
train_ds.imgs = train_samples # Update .imgs as well for compatibility

val_ds = datasets.ImageFolder(val_dir, transform=eval_tfms)
val_ds.samples = val_samples
val_ds.imgs = val_samples # Update .imgs as well for compatibility

# Set the custom loader for the filtered datasets
train_ds.loader = robust_opencv_loader
val_ds.loader = robust_opencv_loader


print("Classes:", train_ds.classes)
print("New Train (valid):", len(train_ds), " Validation (valid):", len(val_ds))

# --- Model, Criterion, Optimizer (re-define just in case) ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# --- DataLoaders ---
# Only create DataLoaders if there are valid samples
if len(train_samples) > 0 and len(val_samples) > 0:
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

    # Define the number of epochs
    num_epochs = 10

    # --- Training loop ---
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()  # Set model to training mode
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_ds)

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * inputs.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total_predictions += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_ds)
        val_accuracy = correct_predictions / total_predictions

        end_time = time.time()
        epoch_duration = end_time - start_time

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

    print("Training complete.")
else:
    print("Not enough valid samples to proceed with training.")


Filtering training images...
Filtering validation images...
Number of valid training samples: 0
Number of valid validation samples: 0
Classes: ['clear', 'cloudy']
New Train (valid): 0  Validation (valid): 0
Using device: cpu
Not enough valid samples to proceed with training.


## 훈련 모델 재시도 1

### 하위 작업:
이미지 로딩 문제를 해결한 후 모델 학습을 다시 시도합니다.


**추론**:
이전 모델 학습 시도가 실패한 이유는 사용자 지정 OpenCV 로더에서 이미지를 로드할 수 없었기 때문입니다. 이는 이미지 파일 읽기에 근본적인 문제가 있음을 나타냅니다. 다음 단계는 실패한 이미지에 대해 보다 자세한 오류 메시지를 제공하도록 강력한 OpenCV 로더를 수정한 다음 데이터 세트를 다시 필터링하고 로드에 실패한 파일의 경로를 기록하여 문제의 범위와 성격을 파악하는 것입니다. 이는 하위 작업의 지침 1과 2를 해결합니다.



In [21]:
import os
import shutil
import random
import cv2
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Log file for failed images
failed_images_log = "failed_images.log"
if os.path.exists(failed_images_log):
    os.remove(failed_images_log) # Clear previous log

# Define a custom loader function using OpenCV with detailed error logging
def robust_opencv_loader_with_logging(path):
    try:
        # Read image with OpenCV
        img = cv2.imread(path, cv2.IMREAD_UNCHANGED)

        # Check if image reading was successful
        if img is None:
            with open(failed_images_log, "a") as f:
                f.write(f"Failed to load image (cv2.imread returned None): {path}\n")
            return None

        # OpenCV reads images as BGR, convert to RGB
        if len(img.shape) == 3:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif len(img.shape) == 2: # Grayscale image
             img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB) # Convert grayscale to RGB by duplicating channels

        # Handle alpha channel if present (more than 3 channels)
        if img.shape[-1] > 3:
            img = img[:, :, :3] # Keep only the first 3 channels (RGB)

        # Convert numpy array to PIL Image
        img = Image.fromarray(img)

        return img
    except Exception as e:
        with open(failed_images_log, "a") as f:
            f.write(f"Error loading image {path} with OpenCV: {e}\n")
        return None


# --- Dataset Creation with Filtering and Logging ---
IMG_SIZE = 224
BATCH = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_dir = "data/classify/new_train"
val_dir = "data/classify/val"

# Load datasets and filter out unreadable images, logging failures
train_ds_full = datasets.ImageFolder(train_dir)
val_ds_full = datasets.ImageFolder(val_dir)

# Filter function to check if an image can be loaded with logging
def is_valid_image_with_logging(filepath, loader):
    img = loader(filepath)
    return img is not None

# Get valid samples and log failed ones
print("Filtering training images and logging failures...")
train_samples = [(path, label) for path, label in train_ds_full.samples if is_valid_image_with_logging(path, robust_opencv_loader_with_logging)]
print("Filtering validation images and logging failures...")
val_samples = [(path, label) for path, label in val_ds_full.samples if is_valid_image_with_logging(path, robust_opencv_loader_with_logging)]

# Print the number of valid samples
print(f"Number of valid training samples: {len(train_samples)}")
print(f"Number of valid validation samples: {len(val_samples)}")

# Check the log file
if os.path.exists(failed_images_log):
    with open(failed_images_log, "r") as f:
        failed_count = len(f.readlines())
    print(f"Number of failed images logged: {failed_count}")
else:
    print("No failed images logged.")

# Create new datasets with only valid samples and apply transforms
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
train_ds.samples = train_samples
train_ds.imgs = train_samples # Update .imgs as well for compatibility

val_ds = datasets.ImageFolder(val_dir, transform=eval_tfms)
val_ds.samples = val_samples
val_ds.imgs = val_samples # Update .imgs as well for compatibility

# Set the custom loader for the filtered datasets
train_ds.loader = robust_opencv_loader_with_logging
val_ds.loader = robust_opencv_loader_with_logging


print("Classes:", train_ds.classes)
print("New Train (valid):", len(train_ds), " Validation (valid):", len(val_ds))

# --- Model, Criterion, Optimizer (re-define just in case) ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# --- DataLoaders and Training Loop (only if valid samples exist) ---
if len(train_samples) > 0 and len(val_samples) > 0:
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

    # Define the number of epochs
    num_epochs = 10

    # --- Training loop ---
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()  # Set model to training mode
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_ds)

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * inputs.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total_predictions += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_ds)
        val_accuracy = correct_predictions / total_predictions

        end_time = time.time()
        epoch_duration = end_time - start_time

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

    print("Training complete.")
else:
    print("Not enough valid samples to proceed with training.")
    # Address instructions 3, 4, 5, 6, 8 here based on the log file
    print(f"\nExamining '{failed_images_log}' for patterns in failed images.")
    if os.path.exists(failed_images_log):
        with open(failed_images_log, "r") as f:
            failed_logs = f.readlines()

        if failed_logs:
            print(f"Found {len(failed_logs)} entries in the log.")
            # Simple analysis of error types and file extensions
            error_types = {}
            file_extensions = {}
            for log_entry in failed_logs:
                if "Error loading image" in log_entry:
                    error_detail = log_entry.split("Error loading image")[1].split("with OpenCV:")[1].strip()
                    error_type = error_detail.split(":")[0].strip() # Basic attempt to get error type
                    error_types[error_type] = error_types.get(error_type, 0) + 1
                elif "cv2.imread returned None" in log_entry:
                     error_types["cv2.imread returned None"] = error_types.get("cv2.imread returned None", 0) + 1

                file_path_match = re.search(r"data/classify/[^:]+:\s(.*)", log_entry) # More robust regex to capture file path
                if file_path_match:
                    file_path = file_path_match.group(1).strip()
                    ext = os.path.splitext(file_path)[1].lower()
                    file_extensions[ext] = file_extensions.get(ext, 0) + 1


            print("\nError Type Distribution:")
            for err_type, count in error_types.items():
                print(f"- {err_type}: {count}")

            print("\nFile Extension Distribution among failed images:")
            for ext, count in file_extensions.items():
                print(f"- {ext}: {count}")

            # Based on the analysis, if it's a specific error or extension,
            # one could investigate further (manual inspection, alternative libraries).
            # Since all images failed, it suggests a broad issue, likely with the TIFF format
            # and OpenCV's ability to read this specific type of TIFF file.
            print("\nConclusion based on log analysis:")
            print("All images failed to load. This suggests a widespread issue with the image format or structure.")
            print("Manual inspection of a few failed files and investigating alternative libraries for TIFF (e.g., GDAL) is recommended.")
        else:
            print("Log file is empty, which is unexpected given the 0 valid samples.")


Filtering training images and logging failures...
Filtering validation images and logging failures...
Number of valid training samples: 0
Number of valid validation samples: 0
Number of failed images logged: 1098
Classes: ['clear', 'cloudy']
New Train (valid): 0  Validation (valid): 0
Using device: cpu
Not enough valid samples to proceed with training.

Examining 'failed_images.log' for patterns in failed images.
Found 1098 entries in the log.

Error Type Distribution:
- cv2.imread returned None: 1098

File Extension Distribution among failed images:

Conclusion based on log analysis:
All images failed to load. This suggests a widespread issue with the image format or structure.
Manual inspection of a few failed files and investigating alternative libraries for TIFF (e.g., GDAL) is recommended.


**추론**:
이전 단계에서 OpenCV를 사용하여 모든 이미지를 로드하지 못했음을 확인했으며, 로그 파일에 실패 이유가 일관되게 `cv2.imread 반환 없음`으로 표시되어 있습니다. 이는 이미지 형식(이전 출력에서 파일 확장자 분포가 비어 있는 것으로 보아 TIFF로 추정되며, 이전 출력에는 확장자가 명시적으로 나열되어 있지 않지만 원본 파일은 `.tif`임)과 OpenCV의 TIFF 리더 간에 호환성 문제가 있음을 강력하게 시사합니다. 분석에서 제안한 대로, 다음 논리적 단계는 GDAL과 같이 TIFF와 같은 지리공간 이미지 형식을 처리하는 것으로 알려진 대체 라이브러리를 사용해 보는 것입니다. 이렇게 하면 대체 로딩 라이브러리를 조사하고 데이터 세트 로딩 프로세스를 업데이트할 준비를 함으로써 지침 5와 6을 해결할 수 있습니다.



In [22]:
import os
import shutil
import random
# import cv2 # Already tried and failed, no need to import again
from PIL import Image
import numpy as np
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim
import time
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
try:
    from osgeo import gdal
except ImportError:
    print("GDAL is not installed. Installing GDAL...")
    # Use pip to install GDAL. This might require system dependencies.
    # This is a common approach in Colab/Jupyter, but might need
    # additional system-level installs depending on the environment.
    !pip install GDAL==$(gdal-config --version | awk -F'[.]' '{print $1"."$2}')
    try:
        from osgeo import gdal
        print("GDAL installed successfully.")
    except ImportError:
        print("GDAL installation failed. Cannot proceed with GDAL loader.")
        gdal = None # Set gdal to None if installation fails

# Log file for failed images
failed_images_log = "failed_images.log"
if os.path.exists(failed_images_log):
    os.remove(failed_images_log) # Clear previous log

# Define a custom loader function using GDAL with detailed error logging
def gdal_loader_with_logging(path):
    if gdal is None:
        return None # Cannot use GDAL if not installed

    try:
        # Open image with GDAL
        dataset = gdal.Open(path, gdal.GA_ReadOnly)
        if dataset is None:
            with open(failed_images_log, "a") as f:
                f.write(f"Failed to open image (gdal.Open returned None): {path}\n")
            return None

        # Read the image data as a numpy array
        # Assuming a standard multi-band image (e.g., RGB or grayscale)
        # Read all bands
        bands_data = []
        for i in range(1, dataset.RasterCount + 1):
            band = dataset.GetRasterBand(i)
            bands_data.append(band.ReadAsArray())

        # Stack bands to create a single numpy array (Height x Width x Channels)
        img_np = np.stack(bands_data, axis=-1)

        # Handle potential different data types (e.g., uint16, float32)
        # Convert to uint8 for PIL compatibility, scaling if necessary
        if img_np.dtype != np.uint8:
            # Simple scaling for demonstration; a more robust approach might
            # involve examining min/max values or using specific driver options
            if np.issubdtype(img_np.dtype, np.integer):
                 # Scale integer types to 0-255
                img_np = (img_np / np.max(img_np) * 255).astype(np.uint8)
            elif np.issubdtype(img_np.dtype, np.floating):
                # Scale float types (assuming 0-1 range or similar)
                img_np = (img_np * 255).astype(np.uint8)


        # Convert numpy array to PIL Image
        # Handle grayscale vs color
        if img_np.shape[-1] == 1:
            img = Image.fromarray(img_np.squeeze(), 'L') # Grayscale
            img = img.convert('RGB') # Convert grayscale to RGB for consistency
        elif img_np.shape[-1] == 3:
             img = Image.fromarray(img_np, 'RGB')
        elif img_np.shape[-1] > 3:
             # If more than 3 channels, assume first 3 are RGB and take them
             img = Image.fromarray(img_np[:,:,:3], 'RGB')
        else:
             with open(failed_images_log, "a") as f:
                f.write(f"Unsupported number of channels ({img_np.shape[-1]}): {path}\n")
             return None


        return img
    except Exception as e:
        with open(failed_images_log, "a") as f:
            f.write(f"Error loading image {path} with GDAL: {e}\n")
        return None


# --- Dataset Creation with Filtering and Logging (using GDAL loader) ---
IMG_SIZE = 224
BATCH = 8

train_tfms = transforms.Compose([
    transforms.RandomResizedCrop(IMG_SIZE),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

train_dir = "data/classify/new_train"
val_dir = "data/classify/val"

# Load datasets and filter out unreadable images, logging failures
train_ds_full = datasets.ImageFolder(train_dir)
val_ds_full = datasets.ImageFolder(val_dir)

# Filter function to check if an image can be loaded with logging
def is_valid_image_with_logging(filepath, loader):
    img = loader(filepath)
    return img is not None

# Get valid samples and log failed ones using the GDAL loader
print("Filtering training images and logging failures using GDAL...")
train_samples = [(path, label) for path, label in train_ds_full.samples if is_valid_image_with_logging(path, gdal_loader_with_logging)]
print("Filtering validation images and logging failures using GDAL...")
val_samples = [(path, label) for path, label in val_ds_full.samples if is_valid_image_with_logging(path, gdal_loader_with_logging)]

# Print the number of valid samples
print(f"Number of valid training samples: {len(train_samples)}")
print(f"Number of valid validation samples: {len(val_samples)}")

# Check the log file
if os.path.exists(failed_images_log):
    with open(failed_images_log, "r") as f:
        failed_logs = f.readlines()

    if failed_logs:
        print(f"Number of failed images logged: {len(failed_logs)}")
        # Simple analysis of error types and file extensions
        error_types = {}
        file_extensions = {}
        for log_entry in failed_logs:
            # Extract error type
            if "Error loading image" in log_entry:
                error_detail_match = re.search(r"with GDAL:\s(.*)", log_entry)
                if error_detail_match:
                    error_detail = error_detail_match.group(1).strip()
                    error_types[error_detail] = error_types.get(error_detail, 0) + 1
            elif "gdal.Open returned None" in log_entry:
                error_types["gdal.Open returned None"] = error_types.get("gdal.Open returned None", 0) + 1
            elif "Unsupported number of channels" in log_entry:
                 error_types["Unsupported number of channels"] = error_types.get("Unsupported number of channels", 0) + 1


            # Extract file path and extension
            file_path_match = re.search(r":\s(.*)", log_entry)
            if file_path_match:
                file_path = file_path_match.group(1).strip()
                ext = os.path.splitext(file_path)[1].lower()
                file_extensions[ext] = file_extensions.get(ext, 0) + 1


        print("\nError Type Distribution (GDAL):")
        for err_type, count in error_types.items():
            print(f"- {err_type}: {count}")

        print("\nFile Extension Distribution among failed images (GDAL):")
        for ext, count in file_extensions.items():
            print(f"- {ext}: {count}")

    else:
        print("Log file is empty. All images loaded successfully with GDAL.")

# Create new datasets with only valid samples and apply transforms
train_ds = datasets.ImageFolder(train_dir, transform=train_tfms)
train_ds.samples = train_samples
train_ds.imgs = train_samples # Update .imgs as well for compatibility

val_ds = datasets.ImageFolder(val_dir, transform=eval_tfms)
val_ds.samples = val_samples
val_ds.imgs = val_samples # Update .imgs as well for compatibility

# Set the custom loader for the filtered datasets to the GDAL loader
train_ds.loader = gdal_loader_with_logging
val_ds.loader = gdal_loader_with_logging


print("Classes:", train_ds.classes)
print("New Train (valid with GDAL):", len(train_ds), " Validation (valid with GDAL):", len(val_ds))

# --- Model, Criterion, Optimizer (re-define just in case) ---
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load a pre-trained ResNet18 model
# Using ResNet18_Weights.DEFAULT is recommended for getting the best available weights
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer to have 2 output features (for clear and cloudy)
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Move the model to the selected device
model = model.to(device)

# Define the loss function
criterion = nn.CrossEntropyLoss()

# Define the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# --- DataLoaders and Training Loop (only if valid samples exist) ---
if len(train_samples) > 0 and len(val_samples) > 0:
    train_loader = DataLoader(train_ds, batch_size=BATCH, shuffle=True, num_workers=2)
    val_loader = DataLoader(val_ds, batch_size=BATCH, shuffle=False, num_workers=2)

    # Define the number of epochs
    num_epochs = 10

    # --- Training loop ---
    print("\nStarting training loop...")
    for epoch in range(num_epochs):
        start_time = time.time()
        model.train()  # Set model to training mode
        running_loss = 0.0

        for i, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            # Zero the parameter gradients
            optimizer.zero_grad()

            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_ds)

        # Validation phase
        model.eval()  # Set model to evaluation mode
        val_running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = model(inputs)
                loss = criterion(outputs, labels)

                val_running_loss += loss.item() * inputs.size(0)

                _, predicted = torch.max(outputs.data, 1)
                total_predictions += labels.size(0)
                correct_predictions += (predicted == labels).sum().item()

        val_loss = val_running_loss / len(val_ds)
        val_accuracy = correct_predictions / total_predictions

        end_time = time.time()
        epoch_duration = end_time - start_time

        print(f"Epoch {epoch+1}/{num_epochs} - Train Loss: {epoch_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}, Time: {epoch_duration:.2f}s")

    print("Training complete.")
else:
    print("Not enough valid samples to proceed with training after attempting GDAL loading.")
    # If GDAL also failed for all images, document the issue and finish
    print("\nGDAL also failed to load all images. The dataset appears to have widespread image loading issues.")
    print("Further investigation into the specific TIFF file structure and potential data corruption is required.")
    print("The task cannot be completed with the current data loading capabilities.")


Filtering training images and logging failures using GDAL...


/usr/local/lib/python3.12/dist-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(
/tmp/ipython-input-2680738158.py:80: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(img_np[:,:,:3], 'RGB')


Filtering validation images and logging failures using GDAL...
Number of valid training samples: 878
Number of valid validation samples: 220
Classes: ['clear', 'cloudy']
New Train (valid with GDAL): 878  Validation (valid with GDAL): 220
Using device: cpu

Starting training loop...
Epoch 1/10 - Train Loss: 0.4195, Val Loss: 0.1735, Val Accuracy: 0.9273, Time: 299.95s
Epoch 2/10 - Train Loss: 0.2271, Val Loss: 5.4235, Val Accuracy: 0.6273, Time: 290.98s
Epoch 3/10 - Train Loss: 0.3021, Val Loss: 1.3424, Val Accuracy: 0.7182, Time: 282.71s
Epoch 4/10 - Train Loss: 0.2160, Val Loss: 0.2873, Val Accuracy: 0.8955, Time: 281.24s
Epoch 5/10 - Train Loss: 0.2613, Val Loss: 0.6499, Val Accuracy: 0.8773, Time: 288.03s
Epoch 6/10 - Train Loss: 0.1544, Val Loss: 1.1784, Val Accuracy: 0.7182, Time: 284.78s
Epoch 7/10 - Train Loss: 0.1898, Val Loss: 0.5023, Val Accuracy: 0.8227, Time: 283.45s
Epoch 8/10 - Train Loss: 0.1741, Val Loss: 0.3889, Val Accuracy: 0.8500, Time: 279.95s
Epoch 9/10 - Train Lo

## 모델 평가

### 하위 작업:
학습된 모델을 유효성 검사 세트에서 평가하여 성능을 평가합니다.


## 요약:

### 데이터 분석 주요 결과

* 초기 테스트 데이터 세트에는 549개의 선명한 이미지와 549개의 흐린 이미지가 균형 있게 포함되었습니다.
* 테스트 데이터 세트를 80/20 비율로 새로운 학습 및 검증 세트로 분할한 후, 새로운 학습 세트에는 선명한 이미지 439개와 흐린 이미지 439개가 포함되었고, 검증 세트에는 선명한 이미지 110개와 흐린 이미지 110개가 포함되었습니다.
* 모든 이미지에서 기본 PIL 로더와 OpenCV 로더를 사용하여 이미지 데이터를 로드하려고 시도했지만 실패했으며, 이는 이미지 형식(아마도 TIFF) 및 이러한 라이브러리에 특정 문제가 있음을 시사합니다.
* 이미지 로딩을 위해 GDAL 라이브러리로 전환하자 878개의 훈련 이미지와 220개의 검증 이미지가 성공적으로 로드되어 훈련 프로세스를 진행할 수 있게 되었습니다.
* 훈련된 ResNet18 모델은 10회 훈련 후 약 93.64%의 검증 정확도를 달성했습니다.

### 인사이트 또는 다음 단계

* PIL 및 OpenCV와 같은 표준 라이브러리로 로드하지 못하게 하는 TIFF 파일의 특정 구조 또는 잠재적 문제를 조사합니다.
* 하이퍼파라미터 튜닝 또는 더 복잡한 아키텍처 사용과 같은 모델 학습의 추가 최적화를 통해 분류 정확도를 개선하는 방안을 고려하세요.



# 추론 단계

# 작업
선택한 셀의 코드를 수정하여 "/content/data/NUAA_CR4L8_9"에서 데이터를 올바르게 로드하고, 학습된 모델의 가중치를 저장하고, 100개의 샘플 이미지에 대해 추론을 수행합니다. 이미지를 흐린 경우 'y_cloud', 맑은 경우 'n_cloud'로 분류하여 새로 만든 폴더에 저장합니다.

## 모델 가중치 저장

### 하위 작업:
학습된 모델의 상태 사전을 파일에 저장합니다.


**추론**:
학습된 모델의 상태 사전을 지시에 따라 파일에 저장합니다.




In [23]:
import torch

# Define a file path for saving the model weights
model_save_path = 'image_classification_model.pth'

# Save the state dictionary of the trained model
torch.save(model.state_dict(), model_save_path)

print(f"Model weights saved to {model_save_path}")

Model weights saved to image_classification_model.pth


## 추론 디렉터리 만들기

### 하위 작업:
추론 결과에 대한 분류된 이미지('y_cloud' 및 'n_cloud')를 저장할 디렉터리를 생성합니다.


**추론**:
지침에 따라 추론 결과에 대한 분류된 이미지를 저장할 디렉터리를 만듭니다.



In [24]:
import os

# Define the base directory for saving inference results
inference_results_dir = "inference_results"

# Create the base directory if it doesn't exist
os.makedirs(inference_results_dir, exist_ok=True)

# Define the paths for the 'y_cloud' and 'n_cloud' subdirectories
y_cloud_dir = os.path.join(inference_results_dir, "y_cloud")
n_cloud_dir = os.path.join(inference_results_dir, "n_cloud")

# Create the 'y_cloud' and 'n_cloud' directories if they don't exist
os.makedirs(y_cloud_dir, exist_ok=True)
os.makedirs(n_cloud_dir, exist_ok=True)

# Print messages confirming the creation of these directories
print(f"Created directory: {y_cloud_dir}")
print(f"Created directory: {n_cloud_dir}")

Created directory: inference_results/y_cloud
Created directory: inference_results/n_cloud


## 샘플 이미지 선택

### 하위 작업:
사용 가능한 데이터에서 추론할 수 있는 샘플 이미지 100개를 선택합니다.


**추론**:
필요한 라이브러리를 가져오고, 테스트 이미지의 기본 디렉터리를 정의하고, 해당 디렉토리와 그 하위 디렉터리에 유효한 확장자를 가진 모든 이미지 파일을 나열하고, 목록을 섞고, 처음 100개의 이미지를 선택하고, 개수를 인쇄합니다.



In [25]:
import glob
import random
import os

# Define the base directory where the test images are located.
# Based on the previous data split, the test images are now in data/classify/val
test_base_dir = "data/classify/val"

# Use glob to get a list of all image file paths within the specified test directory and its subdirectories.
# Consider the image file extensions that were successfully loaded during training (.tif).
# We will also include other common image extensions as a precaution, though .tif seems primary.
valid_extensions = ("*.tif", "*.tiff", "*.png", "*.jpg", "*.jpeg", "*.bmp", "*.webp")
image_files = []
for ext in valid_extensions:
    image_files.extend(glob.glob(os.path.join(test_base_dir, "**", ext), recursive=True))

# Shuffle the list of image file paths randomly.
random.shuffle(image_files)

# Select the first 100 file paths from the shuffled list.
# Ensure we don't select more images than available.
sample_images = image_files[:min(100, len(image_files))]

# Print the number of selected sample images to verify the selection.
print(f"Number of sample images selected for inference: {len(sample_images)}")

Number of sample images selected for inference: 100


## 로드 모델 및 가중치

### 하위 작업:
모델 아키텍처와 저장된 가중치를 로드합니다.


**추론**:
필요한 라이브러리 가져오기, 모델 아키텍처 로드, 최종 레이어 수정, 저장된 가중치 로드, 모델을 올바른 장치로 이동하고 평가 모드로 설정하여 하위 작업의 나머지 모든 지침을 처리합니다.



In [26]:
import torch
import torchvision.models as models

# Check for GPU availability and set the device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Load the pre-trained ResNet18 model architecture
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

# Modify the final fully connected layer to have 2 output features
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)

# Define the path to the saved model weights
model_save_path = 'image_classification_model.pth'

# Load the saved model state dictionary
model.load_state_dict(torch.load(model_save_path, map_location=device))

# Move the model to the selected device
model = model.to(device)

# Set the model to evaluation mode
model.eval()

print("Model loaded and set to evaluation mode successfully.")

Using device: cpu
Model loaded and set to evaluation mode successfully.


## 추론 수행

### 하위 작업:
단일 이미지에 대해 추론을 수행하고 그 클래스('clear' 또는 'cloudy')를 예측하는 함수를 작성합니다.



**추론**:
이전에 정의한 GDAL 로더와 모델을 사용하여 단일 이미지에 대해 추론을 수행하는 함수를 정의합니다. 이는 하위 작업의 1~8번 지침을 해결합니다.



In [27]:
# Define the inference function
def predict_image_class(image_path, model):
    # Use the robust_opencv_loader_with_logging function to load the image
    # Although the function name says opencv, it was updated to use GDAL in the previous step
    # We will rename it for clarity in this step if needed, but for now use the existing name
    # and rely on the fact that it uses GDAL.
    # If the previous step renamed it, use the new name. Let's assume it's still robust_opencv_loader_with_logging
    # based on the available context, but the actual implementation uses GDAL.
    # A more robust approach would be to explicitly define a gdal_loader function here.
    # Let's define a dedicated GDAL loader for clarity in the inference function.

    def inference_gdal_loader(path):
        # This is a simplified version of the robust_opencv_loader_with_logging
        # focusing only on successful loading for inference.
        try:
            dataset = gdal.Open(path, gdal.GA_ReadOnly)
            if dataset is None:
                print(f"Could not open image with GDAL during inference: {path}")
                return None

            bands_data = []
            for i in range(1, dataset.RasterCount + 1):
                band = dataset.GetRasterBand(i)
                bands_data.append(band.ReadAsArray())

            img_np = np.stack(bands_data, axis=-1)

            if img_np.dtype != np.uint8:
                 if np.issubdtype(img_np.dtype, np.integer):
                    img_np = (img_np / np.max(img_np) * 255).astype(np.uint8)
                 elif np.issubdtype(img_np.dtype, np.floating):
                    img_np = (img_np * 255).astype(np.uint8)

            if img_np.shape[-1] == 1:
                img = Image.fromarray(img_np.squeeze(), 'L')
                img = img.convert('RGB')
            elif img_np.shape[-1] == 3:
                 img = Image.fromarray(img_np, 'RGB')
            elif img_np.shape[-1] > 3:
                 img = Image.fromarray(img_np[:,:,:3], 'RGB')
            else:
                 print(f"Unsupported number of channels ({img_np.shape[-1]}) during inference: {path}")
                 return None

            return img
        except Exception as e:
            print(f"Error loading image {path} with GDAL during inference: {e}")
            return None


    # Load the image using the inference-specific GDAL loader
    image = inference_gdal_loader(image_path)

    # Handle cases where the image cannot be loaded
    if image is None:
        return None # Indicate failure to load

    # Apply the eval_tfms transformations
    # Ensure eval_tfms is defined in a previous cell and accessible
    try:
        image_tensor = eval_tfms(image)
    except Exception as e:
        print(f"Error applying transformations to image {image_path}: {e}")
        return None


    # Add a batch dimension
    image_tensor = image_tensor.unsqueeze(0)

    # Move the image tensor to the appropriate device
    image_tensor = image_tensor.to(device)

    # Pass the processed image tensor through the loaded model
    with torch.no_grad(): # No need to calculate gradients during inference
        outputs = model(image_tensor)

    # Get the predicted class index
    _, predicted_class_index = torch.max(outputs.data, 1)

    # Return the predicted class label
    # Ensure train_ds.classes is defined and accessible from a previous cell
    # train_ds.classes should contain ['clear', 'cloudy'] based on previous steps
    predicted_class_label = train_ds.classes[predicted_class_index.item()]

    return predicted_class_label

print("Inference function 'predict_image_class' defined.")

Inference function 'predict_image_class' defined.


## 샘플 이미지 분류

### 하위 작업:
선택한 샘플 이미지를 반복하여 필요한 변환을 적용하고 로드된 모델을 사용하여 이미지를 분류합니다.


**추론**:
샘플 이미지를 반복하고, 정의된 함수를 사용하여 각각을 분류하고, 결과를 저장하고, 진행률 메시지를 인쇄합니다.



In [28]:
# List to store classification results
classification_results = []

# Iterate through the sample images and classify them
print(f"Starting inference on {len(sample_images)} sample images...")
for i, image_path in enumerate(sample_images):
    # Predict the class for the current image
    predicted_label = predict_image_class(image_path, model)

    # Store the result (original path and predicted label)
    classification_results.append({"image_path": image_path, "predicted_label": predicted_label})

    # Print progress message
    if (i + 1) % 20 == 0 or (i + 1) == len(sample_images):
        print(f"Processed {i + 1}/{len(sample_images)} images.")

print("Inference complete.")

Starting inference on 100 sample images...


/tmp/ipython-input-4008678731.py:40: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(img_np[:,:,:3], 'RGB')


Processed 20/100 images.
Processed 40/100 images.
Processed 60/100 images.
Processed 80/100 images.
Processed 100/100 images.
Inference complete.


## 기밀 이미지 저장

### 하위 작업:
분류된 이미지를 적절한 추론 결과 디렉터리로 이동하거나 복사합니다(흐린 경우 'y_cloud', 선명한 경우 'n_cloud').


**추론**:
분류 결과를 반복하고 예측된 레이블에 따라 각 이미지를 적절한 대상 디렉터리에 복사합니다.



In [29]:
import os
import shutil

# Ensure inference result directories are defined and exist (from previous steps)
# inference_results_dir = "inference_results" # Defined previously
# y_cloud_dir = os.path.join(inference_results_dir, "y_cloud") # Defined previously
# n_cloud_dir = os.path.join(inference_results_dir, "n_cloud") # Defined previously

# Iterate through the classification results
print("Saving classified images to their respective directories...")
for result in classification_results:
    image_path = result["image_path"]
    predicted_label = result["predicted_label"]

    # Determine destination directory
    if predicted_label == "cloudy":
        dest_dir = y_cloud_dir
    elif predicted_label == "clear":
        dest_dir = n_cloud_dir
    else:
        print(f"Warning: Unknown predicted label '{predicted_label}' for image {image_path}. Skipping.")
        continue

    # Get the base filename
    filename = os.path.basename(image_path)

    # Construct the full destination path
    dest_path = os.path.join(dest_dir, filename)

    # Copy the image file
    try:
        shutil.copy(image_path, dest_path)
        print(f"Copied image {image_path} to {dest_path}")
    except Exception as e:
        print(f"Error copying image {image_path} to {dest_path}: {e}")

print("\nAll classified images have been saved.")

Saving classified images to their respective directories...
Copied image data/classify/val/clear/test_gt_017035_375.tif to inference_results/n_cloud/test_gt_017035_375.tif
Copied image data/classify/val/cloudy/test_cloud_017035_137.tif to inference_results/n_cloud/test_cloud_017035_137.tif
Copied image data/classify/val/cloudy/test_cloud_028031_700.tif to inference_results/y_cloud/test_cloud_028031_700.tif
Copied image data/classify/val/clear/test_gt_028035_682.tif to inference_results/n_cloud/test_gt_028035_682.tif
Copied image data/classify/val/clear/test_gt_028031_647.tif to inference_results/n_cloud/test_gt_028031_647.tif
Copied image data/classify/val/clear/test_gt_017035_793.tif to inference_results/n_cloud/test_gt_017035_793.tif
Copied image data/classify/val/clear/test_gt_028031_462.tif to inference_results/y_cloud/test_gt_028031_462.tif
Copied image data/classify/val/clear/test_gt_017035_132.tif to inference_results/n_cloud/test_gt_017035_132.tif
Copied image data/classify/val

## 요약:

### 데이터 분석 주요 결과

* 학습된 모델의 상태 사전이 `image_classification_model.pth` 파일에 성공적으로 저장되었습니다.
* 분류된 이미지를 저장하기 위해 `inference_results/y_cloud` 및 `inference_results/n_cloud` 디렉터리가 성공적으로 생성되었습니다.
* 테스트 디렉토리 `data/classify/val`에서 총 100개의 무작위 이미지 파일 경로가 추론을 위해 선택되었습니다.
* 사전 학습된 ResNet18 모델을 로드하고, 2등급 분류를 위해 최종 레이어를 수정한 후 저장된 가중치를 모델에 로드했습니다.
* GDAL 기반 방법을 사용하여 개별 이미지를 로드 및 전처리하고 로드된 모델을 사용하여 추론을 수행한 후 예측된 클래스 라벨('clear' 또는 'cloudy')을 반환하는 Python 함수 `predict_image_class`가 정의되었습니다.
* 100개의 샘플 이미지 모두 `predict_image_class` 함수를 사용하여 성공적으로 분류되었고, 그 결과를 저장했습니다.
* 분류된 이미지들은 예측된 레이블에 따라 'inference\_results' 폴더 내의 각각의 'y_cloud' 또는 'n_cloud' 디렉터리에 성공적으로 복사되었습니다.

### 인사이트 또는 다음 단계

* 추론 프로세스가 성공적으로 완료되어 분류된 이미지 세트가 제공됩니다. 이 결과는 보이지 않는 데이터에 대한 모델의 성능을 평가하는 데 사용할 수 있습니다.
* 특정 파일의 문제를 더 잘 진단하려면 추론 프로세스에 보다 강력한 오류 처리 및 로깅 기능을 추가하여 특히 이미지 로드 및 복사를 위한 로깅을 추가하는 것이 좋습니다.


# 테스트(추론) 설정 및 실험

In [32]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
import os
import glob
import shutil
import cv2
from PIL import Image
import numpy as np
from osgeo import gdal # Assuming GDAL is installed and works for your TIFFs

# --- 설정 (Configuration) ---
# 분류되지 않은 이미지가 포함된 디렉터리 경로를 지정합니다.
UNCLASSIFIED_IMAGES_DIR = "path/to/your/unclassified_images" # !! 이 경로를 실제 이미지 경로로 변경하세요 !!

# 저장된 모델 가중치 파일 경로를 정의합니다.
MODEL_SAVE_PATH = 'image_classification_model.pth'

# 추론 결과를 저장할 기본 디렉터리 경로를 정의합니다.
INFERENCE_RESULTS_DIR = "inference_results_unclassified"

# 모델 입력 이미지 크기
IMG_SIZE = 224

# 사용할 모델 지정 (예: 'resnet18', 'resnet50', 'resnet101' 등)
# torchvision.models에 있는 사전 학습된 모델 이름을 사용하세요.
MODEL_NAME = 'resnet18'


# --- 설정 및 초기화 (Setup) ---
# GPU 사용 가능 여부를 확인하고 장치를 설정합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}") # Using device

# 추론 결과를 저장할 디렉터리 ('y_cloud', 'n_cloud')를 생성합니다 (없을 경우).
y_cloud_dir = os.path.join(INFERENCE_RESULTS_DIR, "y_cloud")
n_cloud_dir = os.path.join(INFERENCE_RESULTS_DIR, "n_cloud")
os.makedirs(y_cloud_dir, exist_ok=True)
os.makedirs(n_cloud_dir, exist_ok=True)
print(f"생성된 디렉터리: {y_cloud_dir}") # Created directory
print(f"생성된 디렉터리: {n_cloud_dir}") # Created directory

# 지정된 모델 아키텍처를 로드합니다.
try:
    # 모델 이름을 기반으로 모델 인스턴스를 생성합니다.
    model = getattr(models, MODEL_NAME)(weights=models.get_model_weights(MODEL_NAME).DEFAULT)
    print(f"모델 아키텍처 로드 성공: {MODEL_NAME}") # Model architecture loaded successfully
except AttributeError:
    print(f"오류: 지원되지 않는 모델 이름 '{MODEL_NAME}'입니다. torchvision.models에서 사용 가능한 모델 이름을 확인하세요.") # Error: Unsupported model name
    exit()


# 최종 완전 연결 레이어를 2개의 출력 특징(clear 및 cloudy)으로 수정합니다.
# 모델 아키텍처에 따라 이 레이어의 이름이 다를 수 있습니다 (예: classifier, fc).
# ResNet 계열은 'fc'를 사용합니다. 다른 모델의 경우 해당 모델의 문서를 확인하세요.
if hasattr(model, 'fc'):
    num_ftrs = model.fc.in_features
    model.fc = nn.Linear(num_ftrs, 2)
elif hasattr(model, 'classifier'): # 예: 일부 Vision Transformer 또는 다른 모델
     # classifier가 순차적인 레이어인 경우 마지막 레이어를 변경합니다.
     if isinstance(model.classifier, nn.Sequential):
         num_ftrs = model.classifier[-1].in_features
         model.classifier[-1] = nn.Linear(num_ftrs, 2)
     else: # classifier가 단일 레이어인 경우
         num_ftrs = model.classifier.in_features
         model.classifier = nn.Linear(num_ftrs, 2)
else:
    print("경고: 모델의 최종 레이어(fc 또는 classifier)를 자동으로 찾을 수 없습니다. 수동으로 수정해야 할 수 있습니다.") # Warning: Could not automatically find the final layer
    # 모델 구조를 출력하여 최종 레이어 이름을 확인하세요.
    # print(model)


# 저장된 모델 가중치를 로드합니다.
try:
    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
    print(f"모델 가중치 로드 성공: {MODEL_SAVE_PATH}") # Model weights loaded successfully
except FileNotFoundError:
    print(f"오류: 모델 가중치 파일이 '{MODEL_SAVE_PATH}' 경로에 없습니다. 파일이 존재하는지 확인하세요.") # Error: Model weights file not found
    # 이 오류 발생 시 스크립트 종료
    exit()


# 모델을 설정된 장치(GPU 또는 CPU)로 이동합니다.
model = model.to(device)

# 모델을 평가 모드로 설정합니다 (드롭아웃 등 비활성화).
model.eval()

# 추론을 위한 이미지 변환을 정의합니다 (eval_tfms와 동일).
inference_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# 추론을 위한 GDAL 이미지 로더 함수를 정의합니다.
def inference_gdal_loader(path):
    try:
        # GDAL을 사용하여 이미지를 엽니다.
        dataset = gdal.Open(path, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"추론 중 GDAL로 이미지를 열 수 없습니다: {path}") # Could not open image with GDAL during inference
            return None

        # 이미지 데이터를 NumPy 배열로 읽습니다.
        # 표준 다중 밴드 이미지(예: RGB 또는 그레이스케일)를 가정합니다.
        bands_data = []
        for i in range(1, dataset.RasterCount + 1):
            band = dataset.GetRasterBand(i)
            bands_data.append(band.ReadAsArray())

        # 밴드를 쌓아 단일 NumPy 배열(높이 x 너비 x 채널)을 생성합니다.
        img_np = np.stack(bands_data, axis=-1)

        # 잠재적인 다른 데이터 유형(예: uint16, float32)을 처리합니다.
        # PIL 호환성을 위해 uint8로 변환하고 필요한 경우 스케일링합니다.
        if img_np.dtype != np.uint8:
             if np.issubdtype(img_np.dtype, np.integer):
                 # 정수형을 0-255 범위로 스케일링합니다.
                img_np = (img_np / np.max(img_np) * 255).astype(np.uint8)
             elif np.issubdtype(img_np.dtype, np.floating):
                 # 부동 소수점 형식을 스케일링합니다 (0-1 범위 등을 가정).
                img_np = (img_np * 255).astype(np.uint8)

        # NumPy 배열을 PIL 이미지로 변환합니다.
        # 그레이스케일 vs 컬러를 처리합니다.
        if img_np.shape[-1] == 1:
            img = Image.fromarray(img_np.squeeze(), 'L') # 그레이스케일
            img = img.convert('RGB') # 일관성을 위해 그레이스케일을 RGB로 변환
        elif img_np.shape[-1] == 3:
             img = Image.fromarray(img_np, 'RGB')
        elif img_np.shape[-1] > 3:
             # 3개 이상의 채널이 있는 경우, 처음 3개를 RGB로 간주하고 가져옵니다.
             img = Image.fromarray(img_np[:,:,:3], 'RGB')
        else:
             print(f"추론 중 지원되지 않는 채널 수 ({img_np.shape[-1]}): {path}") # Unsupported number of channels during inference
             return None

        return img
    except Exception as e:
        print(f"추론 중 GDAL로 이미지 로드 오류 {path}: {e}") # Error loading image with GDAL during inference
        return None

# 클래스 이름을 가져옵니다 (훈련 중과 동일한 순서라고 가정).
# 이전 출력에 따르면 클래스는 ['clear', 'cloudy']였습니다.
class_names = ['clear', 'cloudy']


# --- 추론 실행 (Inference) ---
# 분류되지 않은 디렉터리 내의 모든 이미지 파일 목록을 가져옵니다.
unclassified_image_files = []
valid_extensions = ("*.tif", "*.tiff", "*.png", "*.jpg", "*.jpeg", "*.bmp", "*.webp")
for ext in valid_extensions:
    unclassified_image_files.extend(glob.glob(os.path.join(UNCLASSIFIED_IMAGES_DIR, "**", ext), recursive=True))

print(f"\n'{UNCLASSIFIED_IMAGES_DIR}'에서 분류할 이미지 {len(unclassified_image_files)}개를 찾았습니다.") # Found images for classification.

# 분류되지 않은 이미지를 반복하며 분류를 수행합니다.
print("분류 시작...") # Starting classification
classified_count = {"y_cloud": 0, "n_cloud": 0}
for i, image_path in enumerate(unclassified_image_files):
    # 이미지를 로드하고 변환을 적용합니다.
    image = inference_gdal_loader(image_path)

    # 이미지 로드에 실패한 경우 건너뜁니다.
    if image is None:
        continue

    try:
        image_tensor = inference_tfms(image)
    except Exception as e:
        print(f"이미지 변환 적용 오류 {image_path}: {e}") # Error applying transformations to image
        continue


    # 배치 차원을 추가하고 장치로 이동합니다.
    image_tensor = image_tensor.unsqueeze(0).to(device)

    # 추론을 수행합니다.
    with torch.no_grad(): # 추론 중에는 그래디언트 계산이 필요 없습니다.
        outputs = model(image_tensor)

    # 예측된 클래스를 가져옵니다.
    _, predicted_class_index = torch.max(outputs.data, 1)
    predicted_label = class_names[predicted_class_index.item()]

    # 대상 디렉터리를 결정하고 이미지를 저장합니다.
    if predicted_label == "cloudy":
        dest_dir = y_cloud_dir
        classified_count["y_cloud"] += 1
    elif predicted_label == "clear":
        dest_dir = n_cloud_dir
        classified_count["n_cloud"] += 1
    else:
        print(f"경고: 이미지 {image_path}에 대해 알 수 없는 예측 레이블 '{predicted_label}'이 있습니다. 저장을 건너뜁니다.") # Warning: Unknown predicted label
        continue

    # 기본 파일 이름과 대상 경로를 구성합니다.
    filename = os.path.basename(image_path)
    dest_path = os.path.join(dest_dir, filename)

    # 이미지 파일을 복사합니다.
    try:
        shutil.copy(image_path, dest_path)
        # print(f"'{predicted_label}'로 분류된 이미지 {image_path}를 {dest_path}에 저장했습니다.") # 선택 사항: 모든 파일 출력
    except Exception as e:
        print(f"이미지 복사 오류 {image_path}를 {dest_path}로: {e}") # Error copying image

    # 진행 상황 메시지를 출력합니다.
    if (i + 1) % 50 == 0 or (i + 1) == len(unclassified_image_files):
        print(f"처리된 이미지: {i + 1}/{len(unclassified_image_files)}") # Processed images


print("\n분류 완료.") # Classification complete
print(f"결과 저장 위치: {INFERENCE_RESULTS_DIR}") # Saved to
print(f"'y_cloud'로 분류된 이미지: {classified_count['y_cloud']}개") # Classified as 'y_cloud'
print(f"'n_cloud'로 분류된 이미지: {classified_count['n_cloud']}개") # Classified as 'n_cloud'

사용 장치: cpu
생성된 디렉터리: inference_results_unclassified/y_cloud
생성된 디렉터리: inference_results_unclassified/n_cloud
모델 아키텍처 로드 성공: resnet18
모델 가중치 로드 성공: image_classification_model.pth

'path/to/your/unclassified_images'에서 분류할 이미지 0개를 찾았습니다.
분류 시작...

분류 완료.
결과 저장 위치: inference_results_unclassified
'y_cloud'로 분류된 이미지: 0개
'n_cloud'로 분류된 이미지: 0개


## 모델 평가

### 하위 작업:
학습된 모델을 유효성 검사 세트에서 평가하여 성능을 평가합니다.

In [35]:
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import numpy as np
import os
from torchvision import datasets, transforms
import cv2
from PIL import Image
from osgeo import gdal # Assuming GDAL is installed and works for your TIFFs

# Define the GDAL loader function for evaluation with error handling
def inference_gdal_loader(path):
    try:
        dataset = gdal.Open(path, gdal.GA_ReadOnly)
        if dataset is None:
            print(f"경고: 추론 중 GDAL로 이미지를 열 수 없습니다: {path}") # Warning: Could not open image with GDAL during inference
            return None

        bands_data = []
        for i in range(1, dataset.RasterCount + 1):
            band = dataset.GetRasterBand(i)
            bands_data.append(band.ReadAsArray())

        img_np = np.stack(bands_data, axis=-1)

        if img_np.dtype != np.uint8:
             if np.issubdtype(img_np.dtype, np.integer):
                # Scale integer types to 0-255
                max_val = np.max(img_np)
                if max_val > 0:
                    img_np = (img_np / max_val * 255).astype(np.uint8)
                else: # Handle case where all pixel values are 0
                    img_np = np.zeros_like(img_np, dtype=np.uint8)
             elif np.issubdtype(img_np.dtype, np.floating):
                # Scale float types (assuming 0-1 range or similar)
                img_np = (img_np * 255).astype(np.uint8)
             else:
                 print(f"경고: 지원되지 않는 데이터 유형 ({img_np.dtype})입니다. uint8로 변환하지 못했습니다: {path}") # Warning: Unsupported data type
                 return None


        # Convert numpy array to PIL Image
        # Handle grayscale vs color
        if img_np.shape[-1] == 1:
            img = Image.fromarray(img_np.squeeze(), 'L') # Grayscale
            img = img.convert('RGB') # Convert grayscale to RGB for consistency
        elif img_np.shape[-1] == 3:
             img = Image.fromarray(img_np, 'RGB')
        elif img_np.shape[-1] > 3:
             # If more than 3 channels, assume first 3 are RGB and take them
             img = Image.fromarray(img_np[:,:,:3], 'RGB')
        else:
             print(f"경고: 추론 중 지원되지 않는 채널 수 ({img_np.shape[-1]}): {path}") # Warning: Unsupported number of channels during inference
             return None

        return img
    except Exception as e:
        print(f"경고: 추론 중 GDAL로 이미지 로드 오류 {path}: {e}") # Warning: Error loading image with GDAL during inference
        return None


# --- 설정 (Configuration) ---
# 유효성 검사 데이터셋 디렉터리 경로 (이전 단계에서 분할된 데이터 사용)
# 이전 단계에서 'data/classify/val' 경로를 사용했습니다.
VAL_DATA_DIR = "data/classify/val"

# 모델 입력 이미지 크기 (훈련/추론 시 사용한 것과 동일해야 함)
IMG_SIZE = 224

# 배치 크기 (추론 시 메모리 사용량에 맞춰 조절 가능)
BATCH_SIZE = 16 # 평가 시에는 더 큰 배치 크기를 사용할 수 있습니다.

# 사용할 모델 지정 (훈련/추론 시 사용한 것과 동일해야 함)
MODEL_NAME = 'resnet18' # Ensure this matches the model used for training

# 저장된 모델 가중치 파일 경로 (훈련 시 저장한 것과 동일해야 함)
MODEL_SAVE_PATH = 'image_classification_model.pth'


# --- 설정 및 초기화 (Setup) ---
# GPU 사용 가능 여부를 확인하고 장치를 설정합니다.
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"사용 장치: {device}") # Using device

# 유효성 검사 데이터셋에 적용할 이미지 변환을 정의합니다 (훈련 시 eval_tfms와 동일).
eval_tfms = transforms.Compose([
    transforms.Resize(IMG_SIZE+32),
    transforms.CenterCrop(IMG_SIZE),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225]),
])

# 유효성 검사 데이터셋을 로드합니다.
# ImageFolder는 디렉터리 구조를 기반으로 데이터셋을 자동으로 생성합니다.
# 이전에 정의한 GDAL 로더를 사용하도록 설정합니다.
val_ds = datasets.ImageFolder(VAL_DATA_DIR, transform=eval_tfms, loader=inference_gdal_loader) # Use the GDAL loader

# 데이터 로더를 생성합니다.
# num_workers를 줄여서 디버깅을 더 쉽게 할 수 있습니다.
val_loader = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=0) # 평가 시에는 shuffle=False


print(f"유효성 검사 데이터셋 로드 완료: {len(val_ds)}개 이미지") # Validation dataset loaded

# 학습된 모델을 로드합니다.
# 모델 아키텍처는 훈련/추론 시 사용한 것과 동일해야 합니다.
# MODEL_NAME 변수를 사용하여 모델을 지정합니다.
# 저장된 모델 가중치 파일 경로도 동일해야 합니다.
# MODEL_SAVE_PATH 변수를 사용하여 가중치 파일 경로를 지정합니다.
try:
    # 모델 이름을 기반으로 모델 인스턴스를 생성합니다.
    # weights=models.get_model_weights(MODEL_NAME).DEFAULT를 사용하여 사전 학습된 가중치를 로드합니다.
    model = getattr(models, MODEL_NAME)(weights=models.get_model_weights(MODEL_NAME).DEFAULT)
    # 최종 완전 연결 레이어를 2개의 출력 특징으로 수정합니다.
    if hasattr(model, 'fc'):
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, 2)
    elif hasattr(model, 'classifier'):
         if isinstance(model.classifier, nn.Sequential):
             num_ftrs = model.classifier[-1].in_features
             model.classifier[-1] = nn.Linear(num_ftrs, 2)
         else:
             num_ftrs = model.classifier.in_features
             model.classifier = nn.Linear(num_ftrs, 2)
    else:
        print("경고: 모델의 최종 레이어(fc 또는 classifier)를 자동으로 찾을 수 없습니다.")


    # 저장된 모델 가중치를 로드합니다.
    model.load_state_dict(torch.load(MODEL_SAVE_PATH, map_location=device))
    print(f"모델 가중치 로드 성공: {MODEL_SAVE_PATH}") # Model weights loaded successfully

except FileNotFoundError:
    print(f"오류: 모델 가중치 파일이 '{MODEL_SAVE_PATH}' 경로에 없습니다. 파일이 존재하는지 확인하세요.") # Error: Model weights file not found
    exit()
except AttributeError:
    print(f"오류: 지원되지 않는 모델 이름 '{MODEL_NAME}'입니다. torchvision.models에서 사용 가능한 모델 이름을 확인하세요.") # Error: Unsupported model name
    exit()


# 모델을 설정된 장치(GPU 또는 CPU)로 이동합니다.
model = model.to(device)

# 모델을 평가 모드로 설정합니다 (드롭아웃 등 비활성화).
model.eval()

# 클래스 이름을 가져옵니다 (훈련 중과 동일한 순서라고 가정).
# 이전 출력에 따르면 클래스는 ['clear', 'cloudy']였습니다.
class_names = val_ds.classes # 데이터셋에서 직접 클래스 이름을 가져오는 것이 더 안전합니다.


# --- 모델 평가 (Model Evaluation) ---
print("\n모델 평가 시작...") # Starting model evaluation

# 예측 결과와 실제 레이블을 저장할 리스트
all_predictions = []
all_labels = []

# 그래디언트 계산 비활성화 (평가 모드)
with torch.no_grad():
    for inputs, labels in val_loader:
        # 데이터를 장치로 이동
        inputs, labels = inputs.to(device), labels.to(device)

        # 모델 순전파 (Forward pass)
        outputs = model(inputs)

        # 예측된 클래스 인덱스 가져오기
        _, predicted = torch.max(outputs.data, 1)

        # 결과 저장
        all_predictions.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# NumPy 배열로 변환
all_predictions = np.array(all_predictions)
all_labels = np.array(all_labels)

# --- 평가 지표 계산 (Calculate Evaluation Metrics) ---
print("\n평가 지표 계산...") # Calculating evaluation metrics

# 정확도 (Accuracy)
accuracy = accuracy_score(all_labels, all_predictions)
print(f"정확도 (Accuracy): {accuracy:.4f}") # Accuracy

# 정밀도 (Precision) - 'cloudy' 클래스 (긍정 클래스라고 가정)에 대한 정밀도
# pos_label은 긍정 클래스의 인덱스를 지정합니다. 'cloudy'가 1번 인덱스라고 가정합니다.
# 클래스 순서는 ImageFolder가 디렉터리 이름을 알파벳 순서로 정렬하므로 'clear'가 0, 'cloudy'가 1일 가능성이 높습니다.
# val_ds.classes를 확인하여 정확한 인덱스를 사용하세요.
if 'cloudy' in class_names:
    cloudy_index = class_names.index('cloudy')
    precision = precision_score(all_labels, all_predictions, pos_label=cloudy_index, average='binary')
    print(f"정밀도 (Precision) - 'cloudy': {precision:.4f}") # Precision - 'cloudy'
else:
    print("경고: 'cloudy' 클래스를 찾을 수 없어 정밀도를 계산할 수 없습니다.") # Warning: Could not find 'cloudy' class

# 재현율 (Recall) - 'cloudy' 클래스에 대한 재현율
if 'cloudy' in class_names:
    recall = recall_score(all_labels, all_predictions, pos_label=cloudy_index, average='binary')
    print(f"재현율 (Recall) - 'cloudy': {recall:.4f}") # Recall - 'cloudy'
else:
    print("경고: 'cloudy' 클래스를 찾을 수 없어 재현율을 계산할 수 없습니다.") # Warning: Could not find 'cloudy' class


# F1-스코어 (F1-Score) - 'cloudy' 클래스에 대한 F1-스코어
if 'cloudy' in class_names:
    f1 = f1_score(all_labels, all_predictions, pos_label=cloudy_index, average='binary')
    print(f"F1-스코어 (F1-Score) - 'cloudy': {f1:.4f}") # F1-Score - 'cloudy'
else:
    print("경고: 'cloudy' 클래스를 찾을 수 없어 F1-스코어를 계산할 수 없습니다.") # Warning: Could not find 'cloudy' class


# 혼동 행렬 (Confusion Matrix)
# 행: 실제 레이블, 열: 예측 레이블
# 예: 혼동 행렬 [[TN, FP], [FN, TP]]
conf_matrix = confusion_matrix(all_labels, all_predictions)
print("\n혼동 행렬 (Confusion Matrix):") # Confusion Matrix
print(conf_matrix)

# 클래스별 정확도 (Optional)
if len(class_names) == 2:
    tn, fp, fn, tp = conf_matrix.ravel()
    clear_accuracy = tn / (tn + fp) if (tn + fp) > 0 else 0
    cloudy_accuracy = tp / (tp + fn) if (tp + fn) > 0 else 0
    print(f"\n클래스별 정확도:") # Accuracy per class
    print(f" - {class_names[0]} (실제): {clear_accuracy:.4f}") # clear (Actual)
    print(f" - {class_names[1]} (실제): {cloudy_accuracy:.4f}") # cloudy (Actual)


print("\n모델 평가 완료.") # Model evaluation complete

사용 장치: cpu
유효성 검사 데이터셋 로드 완료: 220개 이미지
모델 가중치 로드 성공: image_classification_model.pth

모델 평가 시작...


/tmp/ipython-input-278410367.py:51: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  img = Image.fromarray(img_np[:,:,:3], 'RGB')



평가 지표 계산...
정확도 (Accuracy): 0.9364
정밀도 (Precision) - 'cloudy': 0.9800
재현율 (Recall) - 'cloudy': 0.8909
F1-스코어 (F1-Score) - 'cloudy': 0.9333

혼동 행렬 (Confusion Matrix):
[[108   2]
 [ 12  98]]

클래스별 정확도:
 - clear (실제): 0.9818
 - cloudy (실제): 0.8909

모델 평가 완료.


### 결과 이미지 다운로드

In [36]:
import shutil
import os

# 압축할 폴더 경로
folder_to_compress = '/content/inference_results'
# 생성될 zip 파일 경로 및 이름
output_zip_file = '/content/inference_results.zip'

# 폴더를 zip 파일로 압축
try:
    shutil.make_archive(output_zip_file.replace('.zip', ''), 'zip', folder_to_compress)
    print(f"'{folder_to_compress}' 폴더가 '{output_zip_file}'으로 성공적으로 압축되었습니다.")
    print("파일 탐색기에서 이 파일을 찾아 다운로드할 수 있습니다.")
except FileNotFoundError:
    print(f"오류: '{folder_to_compress}' 폴더를 찾을 수 없습니다. 경로를 확인해주세요.")
except Exception as e:
    print(f"압축 중 오류가 발생했습니다: {e}")

'/content/inference_results' 폴더가 '/content/inference_results.zip'으로 성공적으로 압축되었습니다.
파일 탐색기에서 이 파일을 찾아 다운로드할 수 있습니다.
